# Max Fine July 2 2024

This notebook serves as an simple example of loading in, and plotting a "raw" `.fil` file, and seeing if my masked channels are in the correct ordering or are reversed

In [1]:
# imports 

import matplotlib.pyplot as plt
from your.formats.pysigproc import SigprocFile
import numpy as np
from your import Writer, Your
from ipywidgets import interact
import os

# to save the plots with the interactive set up
import mplcursors
import mpld3

# data dir
datadir = '/data/frb/2024-06-25/'

# file
pv_file = 'FRB20240209A_PV_Band_2024_06_25_15_27_36.fil'
ph_file = 'FRB20240209A_PH_Band_2024_06_25_15_30_27.fil'
l1_file = 'FRB20240209A_L1_Band_2024_06_25_15_30_27.fil'
l2_file = 'FRB20240209A_L2_Band_2024_06_25_15_30_27.fil'

data_files = [pv_file, ph_file, l1_file, l2_file]


# ignore chan filenames 
# should be located in same dir as this script
p_ignore = 'p_band_master.ignorechans'
l1_ignore = 'l1_band_master.ignorechans'
l2_ignore = 'l2_band_master.ignorechans'


In [2]:
pwd

'/home_local/maxfine/injection_simulations'

### Functions for reading in, and plotting `.fil` files

In [3]:
def get_s(s, tsamp):
    """
    Convert seconds to the number of samples.
    
    Parameters:
    s (float): Time in seconds.
    
    Returns:
    int: Corresponding number of samples.
    """
    return int(s / tsamp)


def make_plot(t_start=30.5, title='Dynamic Spectrum', ignorechan=False, n_seconds=1, quantile1=0.05, 
              quantile2=0.999, bandpass_cor=True, save_path=True):
    """
    Create and display a plot of the dynamic spectrum.
    
    Parameters:
    t_start (float, optional): Start time in seconds. Default is 30.5.
    title (str, optional): Title of the plot. Default is 'Dynamic Spectrum'.
    n_seconds (float, optional): Duration in seconds to plot. Default is 1.
    quantile1 (float, optional): Lower quantile for color scaling. Default is 0.05.
    quantile2 (float, optional): Upper quantile for color scaling. Default is 0.999.
    bandpass_cor (bool, optional): If true, converts for each freq channal data = (data-mean)/STD. Default is True
    """

    #print('ignorechan is ', ignorechan)
    # Extract frequency and time information
    fch1 = fil.fch1  # Frequency of the first channel
    foff = fil.foff  # Frequency offset between channels
    nchans = fil.nchans  # Number of channels
    tsamp = fil.tsamp # Time resolution
    tsamp = fil.native_tsamp()

    # Calculate the number of samples to plot based on the start time and duration
    nsamp = get_s(t_start + n_seconds, tsamp) - get_s(t_start, tsamp)
    nstart = get_s(t_start, tsamp)
    
    # Retrieve the dynamic spectrum data from the .fil file and copy it
    d = fil.get_data(nstart, nsamp).squeeze().copy()
    
    # Bandpass correction: subtract mean and divide by standard deviation
    if bandpass_cor:
        mean_d = np.nanmean(d, axis=1)
        std_d = np.nanstd(d, axis=1)
        print('Shape of mean_d, std_d:', mean_d.shape, std_d.shape)
        
        # Verify shapes
        if mean_d.shape != std_d.shape:
            raise ValueError("Shapes of mean_d and std_d do not match!")
        
        plotdata = (d - mean_d[:, np.newaxis]) / std_d[:, np.newaxis]
    else:
        plotdata = d


    # Mask the bad channels with ignorechan
    if ignorechan:
        # String specifying the ranges of channels to set to NaN
        ranges_str = ignorechan
        
        # Set the specified ranges to NaN
        modified_data = set_nan_ranges(plotdata, ranges_str)
        d = modified_data
            # Bandpass correction: subtract mean and divide by standard deviation
        if bandpass_cor:
            mean_d = np.nanmean(d, axis=0)
            std_d = np.nanstd(d, axis=0)
            plotdata = (d - mean_d) / std_d
        else:
            plotdata = d
        
    
    
    # Create the plot
    fig, ax = plt.subplots()
    
    # Set the labels and title of the plot
    ax.set_xlabel('Time (s)', fontsize=15)
    ax.set_ylabel('Frequency Channel', fontsize=15)
    ax.set_title(title, fontsize=15)
    
    # Display the image with the specified color scaling and colormap
    ax.imshow(
        plotdata,
        extent=[nstart * tsamp, (nstart + nsamp) * tsamp, fil.nchans, 1], 
        origin="upper",
        vmin=np.nanquantile(plotdata, quantile1),
        vmax=np.nanquantile(plotdata, quantile2),
        cmap='turbo'
    )
    

    # Set the aspect ratio of the plot to auto
    ax.set_aspect("auto")

    # Enable interactivity with mplcursors
    mplcursors.cursor(ax)
    
    # save the plot
    if save_path:
        plt.savefig(title + '.png', dpi=300, bbox_inches='tight')
        print(f"Interactive plot saved to cwd")


  

### Functions for reading in the ignorechans files

In [4]:
def read_in_ignore_chan(filename):
    '''This function reads in the three .ignorechans files used as the ignorechanoption in prepsubband '''
    # Get the directory path where the script is located
   # script_dir = os.path.dirname(__file__)

    # Create the full path to the  local file using os.path.join()
    #local_file = os.path.join(script_dir, filename)
    local_file = '/home_local/maxfine/injection_simulations/' + filename
    with open(local_file, 'r') as f:
        for line in f: # there should only be one line
            line = line.strip()
            ignore_chans = line
        return ignore_chans
           # print(ignore_chans)

def parse_ranges(ranges_str):
    """
    Parse a string containing ranges and single indices.
    
    Parameters:
    ranges_str (str): String containing ranges and indices, e.g., '0:10,24,306:319'.
    
    Returns:
    list: List of indices corresponding to the specified ranges and indices.
    """
    indices = []
    for part in ranges_str.split(','):
        if ':' in part:
            start, end = map(int, part.split(':'))
            indices.extend(range(start, end + 1))
        else:
            indices.append(int(part))
    return indices

def set_nan_ranges(data, ranges_str):
    """
    Set specified ranges of channels to NaN in the data array.
    
    Parameters:
    data (numpy.ndarray): 2D array of data.
    ranges_str (str): String containing ranges and indices, e.g., '0:10,24,306:319'.
    
    Returns:
    numpy.ndarray: Modified data array with specified ranges set to NaN.
    """
    indices = parse_ranges(ranges_str)
    data[indices, :] = np.nan
    return data
    


### plot without masking the bad channels

In [5]:


def show_interactive_plot(filename, make_plot, title='test', ignorechan=False,
             t_start=(0, 54, 0.5), n_seconds=(0.5, 10, 1), 
             quantile1=(0, 1, 0.05), quantile2=(0.92, 1, 0.001),
              bandpass_cor=True):
    
        global fil 
        f = filename
        # Read the .fil file

        fil = SigprocFile(datadir + f )
    
        filterbankfile = f

        #print('ignorechan is ', ignorechan)
        # Extract frequency and time information
        fch1 = fil.fch1  # Frequency of the first channel
        foff = fil.foff  # Frequency offset between channels
        nchans = fil.nchans  # Number of channels
        tsamp = fil.tsamp # Time resolution
        tsamp = fil.native_tsamp()
    
        ignorechan = False
        if "l1" in f.lower():
            ignorechan = read_in_ignore_chan(l1_ignore)
        if 'l2' in f.lower():
            ignorechan = read_in_ignore_chan(l2_ignore)
        if '_p' in f.lower():
            ignorechan = read_in_ignore_chan(p_ignore)
        if not ignorechan:
            ignorechan = False
            print(f, "Could not deduce ignorechan file from filename, don't know zapchans")


        
    
        
        # Generate frequency and time arrays
        freqs = fch1 + np.arange(nchans) * foff
        print('nchans ', nchans)
    
        f_lower = fil.fch1 + fil.nchans * fil.foff
        f_upper = fil.fch1
    
        title = f +'\n Dynamic Spectrum'
        interact(make_plot, title=f +'\n Dynamic Spectrum', ignorechan=ignorechan,
                 t_start=t_start, n_seconds=n_seconds, 
                 quantile1=quantile1, quantile2=quantile2,
                  bandpass_cor=bandpass_cor)


In [6]:
title = '\n Dynamic Spectrum'
ignorechan = False
show_interactive_plot(pv_file, make_plot, title=title, ignorechan=ignorechan,
             t_start=(115, 119.810880 , 0.5), n_seconds=(0.5, 10, 1), 
             quantile1=(0, 1, 0.05), quantile2=(0.92, 1, 0.001),
              bandpass_cor=True)

nchans  160


interactive(children=(FloatSlider(value=115.0, description='t_start', max=119.81088, min=115.0, step=0.5), Tex…

In [7]:
# lets look at this file
# FRB20240209A_PV_Band_2024_06_25_15_27_36.fil we get 20 candiates without the mask

In [8]:
# 0:5,142:143,159

In [ ]:
show_interactive_plot(ph_file, make_plot, title=title, ignorechan=ignorechan,
             t_start=(115, 119.810880 , 0.5), n_seconds=(0.5, 10, 1), 
             quantile1=(0, 1, 0.05), quantile2=(0.92, 1, 0.001),
              bandpass_cor=True)

nchans  160


In [ ]:
show_interactive_plot(l1_file, make_plot, title=title, ignorechan=ignorechan,
             t_start=(115, 119.810880 , 0.5), n_seconds=(1, 20, 3), 
             quantile1=(0, 1, 0.05), quantile2=(0.92, 1, 0.001),
              bandpass_cor=True)

In [ ]:
show_interactive_plot(l2_file, make_plot, title=title, ignorechan=ignorechan,
             t_start=(115, 119.810880 , 0.5), n_seconds=(1, 20, 3), 
             quantile1=(0, 1, 0.05), quantile2=(0.92, 1, 0.001),
              bandpass_cor=True)